In [13]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.models import ResNet18_Weights

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
mean, std = [0.4914, 0.4822, 0.4465], [0.247, 0.243, 0.261]
# These values are mostly used by researchers as found to very useful in fast convergence
img_size=224
crop_size = 224

transform = transforms.Compose(
    [
     transforms.Resize(img_size),#, interpolation=torchvision.transforms.InterpolationMode.BICUBIC),
     #transforms.CenterCrop(crop_size),
     transforms.RandomRotation(20),
     transforms.RandomHorizontalFlip(0.1),
     transforms.ColorJitter(brightness=0.1,contrast = 0.1 ,saturation =0.1 ),
     transforms.RandomAdjustSharpness(sharpness_factor = 2, p = 0.1),
     transforms.ToTensor(),
     transforms.Normalize(mean,std),
     transforms.RandomErasing(p=0.75,scale=(0.02, 0.1),value=1.0, inplace=False)])

transformTest = transforms.Compose(
[
    transforms.Resize((img_size,img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)])

In [12]:
# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./CIFAR10', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./CIFAR10', train=False, download=True, transform=transformTest)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
# Load pre-trained ResNet-18 model
resnet18 = models.resnet18(weights=ResNet18_Weights.DEFAULT)
num_ftrs = resnet18.fc.in_features
for param in resnet18.parameters():
	param.requires_grad = False
resnet18.fc = nn.Linear(num_ftrs, 10)  # Modify fully connected layer to have 10 output classes
resnet18.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:
params_to_update = []
for name,param in resnet18.named_parameters():
	if param.requires_grad == True:
		params_to_update.append(param)
		print("\t",name)

	 fc.weight
	 fc.bias


In [16]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
# optimizer = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)

# Training the model
num_epochs = 10
for epoch in range(num_epochs):
    resnet18.train()  # Set model to training mode
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

# Evaluate the model on the test set
correct = 0
total = 0
resnet18.eval()  # Set model to evaluation mode
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = resnet18(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

[1,   100] loss: 2.005
[1,   200] loss: 1.524
[1,   300] loss: 1.352
[1,   400] loss: 1.266
[1,   500] loss: 1.213
[1,   600] loss: 1.187
[1,   700] loss: 1.130
[1,   800] loss: 1.096
[1,   900] loss: 1.087
[1,  1000] loss: 1.086
[1,  1100] loss: 1.042
[1,  1200] loss: 1.001
[1,  1300] loss: 1.008
[1,  1400] loss: 1.034
[1,  1500] loss: 1.007
[2,   100] loss: 0.994
[2,   200] loss: 0.973
[2,   300] loss: 0.987
[2,   400] loss: 0.985
[2,   500] loss: 0.972
[2,   600] loss: 0.964
[2,   700] loss: 0.978
[2,   800] loss: 0.972
[2,   900] loss: 0.980
[2,  1000] loss: 0.987
[2,  1100] loss: 0.922
[2,  1200] loss: 0.955
[2,  1300] loss: 0.972
[2,  1400] loss: 0.976
[2,  1500] loss: 0.954
[3,   100] loss: 0.949
[3,   200] loss: 0.934
[3,   300] loss: 0.942
[3,   400] loss: 0.926
[3,   500] loss: 0.956
[3,   600] loss: 0.931
[3,   700] loss: 0.938
[3,   800] loss: 0.934
[3,   900] loss: 0.922
[3,  1000] loss: 0.954
[3,  1100] loss: 0.925
[3,  1200] loss: 0.904
[3,  1300] loss: 0.950
[3,  1400] 

In [17]:
torch.save(resnet18.state_dict(), 'cifar10_resnet/finetuned.pth')